In [1]:
import os
import shutil
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
import evaluate
import wandb
from huggingface_hub import HfFolder
from wandb.integration.sb3 import WandbCallback
import gzip
import json
import torch.nn as nn
from sklearn.metrics import f1_score, precision_score, recall_score
from torch.nn import CrossEntropyLoss

/nfs/production/literature/santosh_tirunagari/transformers_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-17 02:50:46.542138: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-17 02:50:46.555903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 02:50:46.571086: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been

In [2]:
# Manually set the notebook name for W&B
os.environ["WANDB_NOTEBOOK_NAME"] = "article_classifier.ipynb"

In [3]:
# Initialize WandB
wandb.login()
# !wandb login --relogin

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find article_classifier.ipynb.
wandb: Currently logged in as: tsantosh7 (ebi_literature). Use `wandb login --relogin` to force relogin


True

In [4]:
# Retrieve HF token from environment and authenticate
hf_token = os.getenv('HF_TOKEN')
if hf_token:
    HfFolder.save_token(hf_token)

In [5]:
def cleanup_checkpoints(output_dir, best_model_dir=None, last_model_dir=None):
    """
    Deletes unnecessary model checkpoints created during training.
    Keeps the best model directory and optionally the last model directory.

    :param output_dir: Base directory where the checkpoints are saved.
    :param best_model_dir: Directory of the best model checkpoint.
    :param last_model_dir: Directory of the last model checkpoint.
    """
    for item in os.listdir(output_dir):
        item_path = os.path.join(output_dir, item)
        if os.path.isdir(item_path) and item.startswith("checkpoint"):
            # Check if this path is not the one we want to keep
            if item_path != best_model_dir and item_path != last_model_dir:
                shutil.rmtree(item_path)

In [6]:
def tokenize_and_align_labels(examples, tokenizer, label2id):
    # Tokenize the abstract (input text)
    tokenized_inputs = tokenizer(
        examples["abstract"],
        truncation=True,
        padding='max_length',
        max_length=512
    )
    
    # Convert labels to numeric IDs
    labels = examples["accession_type"]
    
    # Debugging: Print the labels
#     print("Original label:", labels)
    
    # Add the labels to tokenized inputs
    tokenized_inputs["labels"] = labels  # Now it's already numeric

    return tokenized_inputs


In [7]:
def load_jsonlines_to_df(file_path):
    with gzip.open(file_path, 'rt') as gzfile:
        data = [json.loads(line) for line in gzfile]
    
    # Create a DataFrame from the JSON Lines data
    df = pd.DataFrame(data)

    # Ensure 'accession_type' is a list of labels
    if 'accession_type' in df.columns:
        # Modify the 'accession_type' to keep 'metagenomics' and change all others to 'other'
        df['accession_type'] = df['accession_type'].apply(
            lambda labels: ['metagenomics' if 'metagenomics' in labels else 'other']
        )
        
        # Convert multi-label rows into single-label rows
        df_expanded = pd.DataFrame({
            'abstract': df['abstract'].repeat(df['accession_type'].str.len()),  # Repeat each abstract
            'accession_type': [str(label) for sublist in df['accession_type'] for label in sublist]  # Flatten labels to single string
        })
    else:
        raise KeyError("Column 'accession_type' not found in the dataset.")

    return df_expanded


In [8]:
data_folder = "/nfs/production/literature/santosh_tirunagari/BACKUP/work/github/metagenomics/notebooks/"
model_save_path = "/nfs/production/literature/santosh_tirunagari/article_classifier/models/"
pretrained_model = "bioformers/bioformer-8L"

In [9]:
# train_file = os.path.join(data_folder, 'final_multi_label_trainingset.jsonl.gz')
# df = load_jsonlines_to_df(train_file)
# df

In [10]:
# df_expanded = df
# # After transformation, inspect the output
# print(df_expanded['accession_type'].value_counts())

# # Check the unique labels after processing
# unique_labels = df_expanded['accession_type'].unique()
# print("Unique labels after processing:", unique_labels)

# # Mapping "metagenomics" to 1 and "other" to 0
# df_expanded['accession_type'] = df_expanded['accession_type'].apply(lambda x: 1 if x == 'metagenomics' else 0)

# # Check the mapping result
# print(df_expanded['accession_type'].value_counts())


In [11]:
# # Create label mappings
# unique_labels = set(df['accession_type'])
# label2id = {label: idx for idx, label in enumerate(unique_labels)}
# id2label = {idx: label for label, idx in label2id.items()}

# # Print mappings for debugging
# # print("Label to ID mapping:", label2id)

# # Initialize the tokenizer
# tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
    
# # Create label mappings
# unique_labels = set(df['accession_type'])
# label2id = {label: idx for idx, label in enumerate(unique_labels)}
# id2label = {idx: label for label, idx in label2id.items()}

# # Sample 100 examples per class
# df_toy = df.groupby('accession_type').apply(lambda x: x.sample(min(100, len(x)))).reset_index(drop=True)

# # Convert labels to numeric IDs (single-label classification)
# df_toy['accession_type'] = df_toy['accession_type'].apply(lambda x: label2id.get(x, 'Unknown'))

# # Check if any labels were not found
# # print("Labels after conversion:", df_toy['accession_type'].unique())

# # Convert back to Hugging Face Dataset
# dataset = Dataset.from_pandas(df_toy)

# # Split dataset into training and validation
# train_test_split = dataset.train_test_split(test_size=0.1)
# train_dataset = train_test_split['train']
# validation_dataset = train_test_split['test']

# # Tokenize datasets (using the updated function)
# tokenized_train_dataset = train_dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer, label2id), batched=True)
# tokenized_val_dataset = validation_dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer, label2id), batched=True)


In [12]:
# print(tokenized_train_dataset[0])

In [13]:
# Function to compute class weights for single-label classification
def compute_class_weights(labels, num_labels):
    """
    Computes class weights for single-label classification.
    :param labels: A list or array of label IDs.
    :param num_labels: The total number of unique labels.
    :return: A tensor of class weights.
    """
    # Count the occurrence of each label
    class_counts = np.bincount(labels, minlength=num_labels)
    
    # Compute class weights inversely proportional to the frequency of each class
    total_samples = len(labels)
    class_weights = total_samples / (num_labels * class_counts)
    
    # Avoid division by zero in case some classes are missing
    class_weights = np.where(class_counts == 0, 0, class_weights)
    
    # Return class weights as a tensor
    return torch.tensor(class_weights, dtype=torch.float)


class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights  # Save class weights as part of the trainer
    
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        # CrossEntropyLoss for single-label classification with optional class weights
        criterion = CrossEntropyLoss(weight=self.class_weights.to(labels.device) if self.class_weights is not None else None)
        
        loss = criterion(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [14]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def compute_metrics(p):
    logits = p.predictions
    labels = p.label_ids
    
    # Apply softmax to logits to get probabilities for each class
    probs = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
    
    # Get the predicted class by taking the argmax
    preds = np.argmax(logits, axis=-1)
    
    # Ensure labels are binary (single 0 or 1 per example)
    labels = labels.astype(int)
    
#     # Print debugging info
#     print("Logits: ", logits[:5])  # Print first 5 logits
#     print("Probabilities: ", probs[:5])  # Print first 5 probabilities
#     print("Predictions (argmax): ", preds[:5])  # Print first 5 predictions after argmax
#     print("True Labels: ", labels[:5])  # Print first 5 true labels
    
    # Compute binary classification metrics
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="binary")
    precision = precision_score(labels, preds, average="binary")
    recall = recall_score(labels, preds, average="binary")
    
#     print("Accuracy: ", accuracy)
#     print("F1 Score: ", f1)
#     print("Precision: ", precision)
#     print("Recall: ", recall)
    
    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }


# def compute_metrics(p, id2label=None):
#     logits = p.predictions
#     labels = p.label_ids

#     # Apply sigmoid to logits for binary classification
#     preds = (logits > 0).astype(int)
#     labels = labels.astype(int)
#     # Calculate accuracy, F1, precision, and recall
#     accuracy = accuracy_score(labels, preds)
#     f1 = f1_score(labels, preds, average="binary")
#     precision = precision_score(labels, preds, average="binary")
#     recall = recall_score(labels, preds, average="binary")

#     return {
#         "accuracy": accuracy,
#         "f1": f1,
#         "precision": precision,
#         "recall": recall,
#     }

In [15]:
def train_model(data_folder, model_checkpoint, config):

    # Setup WandB if not already initialized
    if not wandb.run:
        print("Initializing WandB...")
        wandb.init(config=config)  # Call init before accessing config
        print("WandB initialized.")
    
    # Now you can access the config safely
    print("Accessing WandB config...")
    learning_rate = wandb.config.learning_rate
    num_train_epochs = wandb.config.num_train_epochs
    train_batch_size = wandb.config.train_batch_size
    eval_batch_size = wandb.config.eval_batch_size
    weight_decay = wandb.config.weight_decay
    print("WandB config accessed successfully.")
    
    # Initialize the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    # Load dataset from file
    train_file = os.path.join(data_folder, 'final_multi_label_trainingset.jsonl.gz')
    df = load_jsonlines_to_df(train_file)

    # Create label mappings
    unique_labels = set(df['accession_type'])
    label2id = {label: idx for idx, label in enumerate(unique_labels)}
    id2label = {idx: label for label, idx in label2id.items()}

    # Print mappings for debugging
    # print("Label to ID mapping:", label2id)

    # Initialize the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

    # Create label mappings
    unique_labels = set(df['accession_type'])
    label2id = {label: idx for idx, label in enumerate(unique_labels)}
    id2label = {idx: label for label, idx in label2id.items()}

    # Sample 100 examples per class
    df_toy = df.groupby('accession_type').apply(lambda x: x.sample(min(100, len(x)))).reset_index(drop=True)

    # Convert labels to numeric IDs (single-label classification)
    df_toy['accession_type'] = df_toy['accession_type'].apply(lambda x: label2id.get(x, 'Unknown'))

    # Check if any labels were not found
    # print("Labels after conversion:", df_toy['accession_type'].unique())

    # Convert back to Hugging Face Dataset
    dataset = Dataset.from_pandas(df_toy)

    # Split dataset into training and validation
    train_test_split = dataset.train_test_split(test_size=0.1)
    train_dataset = train_test_split['train']
    validation_dataset = train_test_split['test']

    # Tokenize datasets (using the updated function)
    tokenized_train_dataset = train_dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer, label2id), batched=True)
    tokenized_val_dataset = validation_dataset.map(lambda x: tokenize_and_align_labels(x, tokenizer, label2id), batched=True)

    # Use sweep name directly for saving the model
    run_name = wandb.run.name  # Use only the sweep name and no fallback to run ID
    model_save_path = os.path.join("./models", run_name)
    os.makedirs(model_save_path, exist_ok=True)

    training_args = TrainingArguments(
        output_dir=model_save_path,
        evaluation_strategy="epoch",  # Evaluate at the end of each epoch
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=train_batch_size,
        per_device_eval_batch_size=eval_batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        load_best_model_at_end=True,
        metric_for_best_model='f1',  # Use the correct metric key
        logging_dir=os.path.join(model_save_path, 'logs'),
        report_to="wandb"
    )

    # Load pre-trained model
    model = AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint,
        num_labels=len(unique_labels)  # Set the number of labels for the model
    )

    
    # Compute class weights using the training labels before tokenization
    train_labels = df_toy['accession_type'].values  # Assuming the labels are in the 'accession_type' column
    class_weights = compute_class_weights(train_labels, num_labels=len(label2id))

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        class_weights=class_weights  # Pass the computed class weights
    )

    # Train model
    trainer.train()

    # Save models
    best_model_path = os.path.join(model_save_path, "best_model")
    trainer.save_model(best_model_path)
    tokenizer.save_pretrained(best_model_path)

    last_model_path = os.path.join(model_save_path, "last_model")
    trainer.save_model(last_model_path)
    tokenizer.save_pretrained(last_model_path)

    # Ensure cleanup function is defined
    cleanup_checkpoints(
        output_dir=model_save_path, 
        best_model_dir=best_model_path, 
        last_model_dir=last_model_path
    )

In [16]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'f1',
        'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {
            'values': [5e-5, 2e-5, 1e-5]
        },
        'train_batch_size': {
            'values': [16]
        },
        'eval_batch_size': {
            'values': [16]
        },
        'num_train_epochs': {
            'values': [10]
        },
        'weight_decay': {
            'values': [0.1, 0.01, 0.001]
        },
        'lr_scheduler_type': {
            'values': ['linear']
        }
    },
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 3,
        'eta': 2
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="single_article_classifier", entity="ebi_literature")
wandb.agent(sweep_id, lambda: train_model(data_folder, pretrained_model, wandb.config))

Create sweep with ID: d35kcwrp
Sweep URL: https://wandb.ai/ebi_literature/single_article_classifier/sweeps/d35kcwrp


wandb: Agent Starting Run: foji6hx8 with config:
wandb: 	eval_batch_size: 16
wandb: 	learning_rate: 1e-05
wandb: 	lr_scheduler_type: linear
wandb: 	num_train_epochs: 10
wandb: 	train_batch_size: 16
wandb: 	weight_decay: 0.01
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find article_classifier.ipynb.


Initializing WandB...


WandB initialized.
Accessing WandB config...
WandB config accessed successfully.


/nfs/production/literature/santosh_tirunagari/transformers_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/nfs/production/literature/santosh_tirunagari/transformers_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1900/1900 [00:00<00:00, 4519.53 examples/s]
/nfs/production/literature/santosh_tirunagari/transformers_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated

...

wandb: Agent Starting Run: gqiggbt1 with config:
wandb: 	eval_batch_size: 16
wandb: 	learning_rate: 1e-05
wandb: 	lr_scheduler_type: linear
wandb: 	num_train_epochs: 10
wandb: 	train_batch_size: 16
wandb: 	weight_decay: 0.01
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find article_classifier.ipynb.


Initializing WandB...


WandB initialized.
Accessing WandB config...
WandB config accessed successfully.


/nfs/production/literature/santosh_tirunagari/transformers_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/nfs/production/literature/santosh_tirunagari/transformers_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1900/1900 [00:00<00:00, 4715.36 examples/s]
/nfs/production/literature/santosh_tirunagari/transformers_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.374200,0.358406,0.853684,0.852442,0.830403,0.875682
2,0.334800,0.344378,0.861579,0.858374,0.847872,0.869138
